### Given a csv file with "address", read it in and add add coordinates in "lat" and "lon" columns using Google Maps Geocoding API
https://developers.google.com/maps/documentation/geocoding/requests-geocoding

In [1]:
import json
import requests
import pandas as pd
from copy import deepcopy
from tqdm import tqdm

In [2]:
# constants
key_json = json.load(open("credentials.json"))
gmaps_key = key_json["key"]
outputFormat = "json"
url = "https://maps.googleapis.com/maps/api/geocode/"

In [10]:
def get_lat_lon_for_address(row):
    address = row["street_and_house_no"] + ' ' + str(row["zipcode"]) + ' ' + row["area"]
    address_http = address.replace(' ', "%20").replace('+', "%2B")
    parameters = f"address={address_http}&key={gmaps_key}"
    request = f"{url}{outputFormat}?{parameters}"
    result = requests.get(request).json()
    location = result["results"][0]["geometry"]["location"]
    return pd.Series([location["lat"], location["lng"]])

In [11]:
df = pd.read_csv("../data/physicians.csv", index_col=0)
tqdm.pandas()
df[["lat", "lon"]] = df.progress_apply(get_lat_lon_for_address, axis=1)
df.head()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2473/2473 [11:26<00:00,  3.60it/s]


,name,expertise,zipcode,area,street_and_house_no,telephone,distance,lat,lon
0,Frau Dr. med. Christa Hutterer,"Fachärztin für Diagnostische Radiologie, Fachä...",80331,München-Altstadt-Lehel,Kaufingerstraße 15,089 / 2 00 01 43 - 50,0.1 km,48.137815,11.571804
1,Herr Dr. med. Josef J. Dohrenbusch,Facharzt für Allgemeinmedizin,80331,München,Kaufingerstraße 12,089 / 24 20 93 98,0.1 km,48.137615,11.573961
2,Frau Dr. med. Heidi Herrmann,Fachärztin für Innere Medizin,80331,München,Altheimer Eck 2,089 / 45 22 81 81,0.1 km,48.137541,11.571022
3,Herr Dr. med. (univ.) Thomas Wendel,Praktischer Arzt,80331,München,Altheimer Eck 10,089 / 89 67 40 20,0.1 km,48.137505,11.570328
4,Frau Dr. med. Sabine Konz,Praktische Ärztin,80331,München,Eisenmannstraße 4,089 / 37 02 97 67,0.2 km,48.137763,11.569028


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2473 entries, 0 to 2472
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   name                 2473 non-null   object 
 1   expertise            2414 non-null   object 
 2   zipcode              2473 non-null   int64  
 3   area                 2473 non-null   object 
 4   street_and_house_no  2473 non-null   object 
 5   telephone            2473 non-null   object 
 6   distance             2473 non-null   object 
 7   lat                  2473 non-null   float64
 8   lon                  2473 non-null   float64
dtypes: float64(2), int64(1), object(6)
memory usage: 193.2+ KB


In [13]:
df.to_csv("../data/physicians_with_coordinates.csv")

In [14]:
len(df)

2473